In [1]:
! pip install underthesea

     |████████████████████████████████| 7.5MB 8.2MB/s 
     |████████████████████████████████| 51kB 6.2MB/s 
     |████████████████████████████████| 1.3MB 37.8MB/s 
     |████████████████████████████████| 747kB 28.6MB/s 
     |████████████████████████████████| 753.2MB 22kB/s 
     |████████████████████████████████| 245kB 34.4MB/s 
     |████████████████████████████████| 6.7MB 37.6MB/s 
     |████████████████████████████████| 1.1MB 42.5MB/s 
     |████████████████████████████████| 890kB 31.9MB/s 
     |████████████████████████████████| 2.9MB 47.4MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp36-none-any.whl size=16171 sha256=7c07d91315080dd7e8fb6314cf2ed6725ab1ad3940bf7c2c9e6fccc705f600d0
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=9544b20385e80fe9d4cfff8febbd86280e7bea40d760ba390ed6f64442c169cc
  Stored in directory:

In [2]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import sys
sys.path.append('/content/drive/My Drive/LDS6_NguyenTanHuy/')
import ML_Model as md
import ML_Phantich as pt
import ML_Transform as mt
# https://github.com/undertheseanlp/UTS2017_BANK

In [12]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
from collections import Counter
import warnings
warnings.filterwarnings("ignore")
import pickle

In [5]:
%cd '/content/drive/My Drive/Colab Notebooks/Phân Loại Category/'

/content/drive/My Drive/Colab Notebooks/Phân Loại Category


In [6]:
df = pd.read_csv("vnexpress_preprocess.csv")

In [ ]:
df.head()

,url,id,category,article,title,description,work_token,work_remove_stop,string_to_wordcloud
0,https://vnexpress.net/tau-cat-linh-ha-dong-cha...,4202780.0,thoi-su,Ngày ông Vũ Hồng Phương Quyền Tổng giám đốc B...,Tàu Cát Linh - Hà Đông chạy thử từ ngày 12/12,Tuyến đường sắt Cát Linh - Hà Đông sẽ vận hành...,"['Ngày', 'ông', 'Vũ Hồng Phương Quyền', 'Tổng ...","['Ngày', 'Vũ Hồng Phương Quyền', 'Tổng giám đố...","Ngày,Vũ Hồng Phương Quyền,Tổng giám đốc,Ban,qu..."
1,https://vnexpress.net/dieu-hanh-tuong-nho-nguo...,4202900.0,thoi-su,Ngày tức Âm lịch cộng đồng người HoaTP Biên H...,Diễu hành tưởng nhớ người có công với đất Đồng...,NaN,"['Ngày', 'tức', 'Âm lịch', 'cộng đồng', 'người...","['Ngày', 'tức', 'Âm lịch', 'cộng đồng', 'HoaTP...","Ngày,tức,Âm lịch,cộng đồng,HoaTP Biên Hòa,tổ c..."
2,https://vnexpress.net/gio-lam-them-se-tang-tu-...,4202671.0,thoi-su,Bộ luật Lao động năm có hiệu lực từ đầu năm s...,Giờ làm thêm sẽ tăng từ năm 2021,"Từ 1/1/2021, làm thêm tối đa tăng lên 40 giờ m...","['Bộ luật', 'Lao động', 'năm', 'có', 'hiệu lực...","['Bộ luật', 'Lao động', 'hiệu lực', 'đầu', 'kh...","Bộ luật,Lao động,hiệu lực,đầu,khống chế,khung,..."
3,https://vnexpress.net/oto-tong-5-xe-may-o-sai-...,4202920.0,thoi-su,Ôtô do tài xế Nhâm Văn Tuấn tuổiquận chạy bên...,Ôtô tông 5 xe máy ở Sài Gòn,Ôtô Kia 4 chỗ chạy trên đường Song Hành Xa Lộ ...,"['Ôtô', 'do', 'tài xế', 'Nhâm Văn Tuấn', 'tuổi...","['Ôtô', 'tài xế', 'Nhâm Văn Tuấn', 'tuổiquận',...","Ôtô,tài xế,Nhâm Văn Tuấn,tuổiquận,chạy,tuyến,M..."
4,https://vnexpress.net/pho-thu-tuong-covid-19-o...,4202843.0,thoi-su,Tại cuộc họp Ban chỉ đạo quốc gia phòng chống...,Phó thủ tướng: 'Covid-19 ở TP HCM cơ bản được ...,"Phó thủ tướng Vũ Đức Đam đánh giá, dù cần tiếp...","['Tại', 'cuộc', 'họp', 'Ban', 'chỉ đạo', 'quốc...","['Tại', 'họp', 'Ban', 'chỉ đạo', 'quốc gia', '...","Tại,họp,Ban,chỉ đạo,quốc gia,phòng chống,Covid..."


In [7]:
df = df.dropna(subset=['string_to_wordcloud'])

In [8]:
def four_pounds_tokenizer(s):
   return s.split(',')

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
mod_tfidf = TfidfVectorizer(tokenizer=four_pounds_tokenizer,max_features=1000)
X = mod_tfidf.fit_transform(df.string_to_wordcloud)

In [13]:
filename = 'tf_idf_topic.sav'
pickle.dump(mod_tfidf, open(filename, 'wb'))

In [14]:
X = X.toarray()

In [15]:
Y = df.category

In [16]:
X_train,X_test,Y_train,Y_test = mt.chia_bo_du_lieu(X,Y,0.3)

In [17]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
np.random.seed(0)

In [18]:
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import  RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

In [19]:
from sklearn import model_selection
from sklearn.model_selection import KFold

In [20]:
pipe = Pipeline([("classifier", RandomForestClassifier())])

search_space = [{"classifier": [LogisticRegression()],"classifier__penalty": ["l1", "l2"],
                },
                {"classifier": [RandomForestClassifier()]
                },
                {"classifier": [DecisionTreeClassifier()]
                },
                {"classifier": [GaussianNB()]
                },
                {"classifier": [MultinomialNB()]
                },
                {"classifier": [BernoulliNB()]
                },
                {"classifier": [svm.SVC()]
                }
]

In [21]:
kfold = KFold(n_splits=5,random_state=42)

In [29]:
clf = GridSearchCV(pipe, search_space, cv=kfold, verbose=0, n_jobs = -1)
best_model = clf.fit(X_train, Y_train)

In [30]:
print(best_model.best_estimator_.get_params()["classifier"])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)


In [ ]:
svm_md = md.SVM_classification(X_train,Y_train)

In [ ]:
print(svm_md.score(X_train,Y_train),svm_md.score(X_test,Y_test))

0.9814483683504693 0.894681960375391


In [ ]:
Y_hat = svm_md.predict(X_test)
pt.danh_gia_mo_hinh_logistic(Y_test,Y_hat)

- Precision and Recall Metric
              precision    recall  f1-score   support

    doi-song       0.91      0.61      0.73        33
     du-lich       0.90      0.60      0.72        47
    giai-tri       0.92      0.88      0.90        65
    giao-duc       0.86      0.92      0.89       187
    khoa-hoc       0.89      0.90      0.89       209
  kinh-doanh       0.91      0.77      0.83        65
  oto-xe-may       0.94      0.94      0.94       196
   phap-luat       0.89      0.94      0.91       222
      so-hoa       0.94      0.93      0.94       221
    suc-khoe       0.82      0.78      0.80        78
    the-gioi       0.88      0.92      0.90       274
    the-thao       0.97      0.95      0.96        88
     thoi-su       0.85      0.88      0.87       233

    accuracy                           0.89      1918
   macro avg       0.90      0.85      0.87      1918
weighted avg       0.90      0.89      0.89      1918

- Confusion Metric
[[ 20   0   3   6   0   1   0 

In [31]:
parameters = {'C':[5,6,7,8,9,10,11,12,13,14,15]}
lm = LogisticRegression()
clf_lm = GridSearchCV(lm, parameters,cv=kfold)

In [32]:
clf_lm.fit(X_train, Y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=False),
             error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [33]:
print(clf_lm.best_estimator_)

LogisticRegression(C=8, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)


In [ ]:
svm_5_md = md.SVM_classification(X_train,Y_train,C=5)

In [34]:
print(clf_lm.score(X_train,Y_train),clf_lm.score(X_test,Y_test))

0.9709581945985941 0.8912855910267472


In [35]:
Y_hat = clf_lm.predict(X_test)
pt.danh_gia_mo_hinh_logistic(Y_test,Y_hat)

- Precision and Recall Metric
              precision    recall  f1-score   support

    doi-song       0.81      0.62      0.70        47
     du-lich       0.84      0.65      0.73        55
    giai-tri       0.89      0.81      0.85        73
    giao-duc       0.81      0.97      0.89       227
    khoa-hoc       0.92      0.90      0.91       277
  kinh-doanh       0.90      0.79      0.84        96
  oto-xe-may       0.96      0.94      0.95       221
   phap-luat       0.92      0.91      0.91       257
      so-hoa       0.88      0.90      0.89       238
    suc-khoe       0.75      0.84      0.79        80
    the-gioi       0.91      0.92      0.91       381
    the-thao       0.95      0.96      0.96        85
     thoi-su       0.87      0.87      0.87       281

    accuracy                           0.89      2318
   macro avg       0.88      0.85      0.86      2318
weighted avg       0.89      0.89      0.89      2318

- Confusion Metric
[[ 29   0   1   7   0   1   0 

In [ ]:
parameters = {'C':[1,2,3,4,5,6,7,8,9]}
svc = svm.SVC()
clf_svc = GridSearchCV(svc, parameters,cv=kfold)

In [ ]:
clf_svc.fit(X, Y)

GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=False),
             error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [1, 2, 3, 4, 5, 6, 7, 8, 9]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [ ]:
print(clf_svc.best_estimator_)

SVC(C=4, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [ ]:
svm_4_md = md.SVM_classification(X_train,Y_train,C=4)

In [ ]:
print(svm_4_md.score(X_train,Y_train),svm_4_md.score(X_test,Y_test))

0.9921770227983907 0.8988529718456726


In [ ]:
Y_hat = svm_4_md.predict(X_test)
pt.danh_gia_mo_hinh_logistic(Y_test,Y_hat)

- Precision and Recall Metric
              precision    recall  f1-score   support

    doi-song       0.80      0.61      0.69        33
     du-lich       0.85      0.60      0.70        47
    giai-tri       0.92      0.94      0.93        65
    giao-duc       0.87      0.92      0.89       187
    khoa-hoc       0.89      0.89      0.89       209
  kinh-doanh       0.88      0.82      0.85        65
  oto-xe-may       0.94      0.94      0.94       196
   phap-luat       0.90      0.94      0.92       222
      so-hoa       0.95      0.93      0.94       221
    suc-khoe       0.81      0.79      0.80        78
    the-gioi       0.90      0.91      0.90       274
    the-thao       0.97      0.95      0.96        88
     thoi-su       0.88      0.90      0.89       233

    accuracy                           0.90      1918
   macro avg       0.89      0.86      0.87      1918
weighted avg       0.90      0.90      0.90      1918

- Confusion Metric
[[ 20   0   3   6   0   1   0 

In [ ]:
import pickle
import joblib

In [38]:
filename = 'clf_lm.sav'
pickle.dump(clf_lm, open(filename, 'wb'))

In [36]:
filename = 'clf_lm.sav'
joblib.dump(log_c8, filename)

NameError: ignored

In [ ]:
from gensim.models import Word2Vec